# Clasificador de emails de Thunderbird a .csv

## Objetivo
Leer los mensajes diarios de `yum update` de las máquinas, y en base a eso crear una lista en un archivo .csv, con un "ok" si no ha habido problemas con la actualización o una x en caso de haberlos o no recibir correo.

## Organización
El programa va a tener dos partes claramente diferenciadas:
1. Análisis de los mensajes
2. Escritura del resultado en un archivo .csv

Para más información, leer el README

## 0. Preparación

Para la tarea que tenemos entre manos necesitamos el módulo pyexcel. Pero hemos de definir los objetos correspondientes al problema entre manos, a saber: número de máquinas, lista con sus nombres (ordenada), lista inicializada con "x", y mensajes de no-error.

In [10]:
#Inputs (en línea de comandos en caso de script.py)
sysargv1 = "18" #día
sysargv2 = "08" #mes
sysargv3 = "--show"

num_to_month = {"01": "Enero", "02": "Febrero", "03": "Marzo", "04": "Abril", "05": "Mayo", "06": "Junio", "07": "Julio", "08": "Agosto", "09": "Septiembre", "10": "Octubre", "11": "Noviembre", "12": "Diciembre" }
month = num_to_month[sysargv2]
todaycolumn = int(sysargv1) #la columna es realmente la sysargv1 + 1 (la primera contiene los servidores). Pero como Python empieza en 0 las listas... 

In [11]:
okkeys = ["No se han seleccionando paquetes para ser actualizados","No packages marked for update", "¡Listo!", "Complete!"] #el primer mensaje de error tiene una errata, sí.

In [12]:
import csv
import pyexcel as pe

In [13]:
xlsx_file = "updates.ods"
sheet = pe.get_book(file_name=xlsx_file)[month]
arrayeet = sheet.get_array()

In [14]:
listservers = []
for i in range(1,len(arrayeet)-3): #quitar las filas extra añadidas.
    listservers.append(arrayeet[i][0].lower())

numservers = len(listservers)
okxlist = ['x'] * numservers

#asegurarse de haber escogido bien los servidores:
if not numservers == 677 or not listservers[0] == "abedul.ccc.uam.es" or not listservers[-1] == "zeus.ccc.uam.es":
    raise ValueError('Algo ha salido mal... la lista de servidores no es correcta.')

En la lista de máquinas, gran parte de ellas no se cuentan a la hora de revisar estos correos porque no los mandan por defecto. Esto puede deberse a que  estén de baja, que sean sondas o SAIs que no mandan mensajes, etc. Vamos a eliminarlos de la lista de "x"

In [15]:
nomsglist = []
for i in range(1,len(arrayeet)-3):
    nomsglist.append(False if not arrayeet[i][todaycolumn - 1] else True) #coger los espacios en blanco del día anterior

numservers = len(listservers)
okxlist = ['x'] * numservers

okxlist =  [a * b for a, b in zip(okxlist,nomsglist)]

# 1. Análisis de los mensajes

Los mensajes se encuentran en un archivo (que Thunderbird considera un directorio de mensajes) dentro de un directorio local creado y usado por Thunderbird.

In [16]:
directorio = "/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/YUMtotal.sbd/"
archivo = sysargv1 + sysargv2 #modificar con el archivo pertinente, o "prueba1"

path = directorio + archivo

In [17]:
print(path)

/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/YUMtotal.sbd/1808


Ahora abriremos el archivo (con codificación "iso-8859-1", como podemos ver al ejecutar en una shell `file -i archivo`), que contiene cada uno de los mensajes de esa carpeta, y los separaremos en elementos de una lista, para luego iterar sobre ellos. Para efectuar esta separación, hemos de darnos cuenta de que en cada archivo de mensajes del directorio, los mensajes empiezan con una línea como por ejemplo `From - Wed Jul 29 15:16:48 2020`. Por tanto usaremos como separador `From -`

In [18]:
with open(path, 'r', encoding='iso-8859-1') as file3:
    messages = file3.read().replace('\n', '')

messlist = messages.split("From -")[1:]

Ahora que tenemos los mensajes, hemos de obtener una lista con los remitentes de cada uno de ellos, a la vez que comprobamos si tienen errores o no.

In [19]:
errlist = []
for mess in messlist:
    sender = mess.split("<root@")[1].split(">Received:")[0]
    num = listservers.index(sender)
    for oks in okkeys:
        err = 0
        if oks in mess:
            okxlist[num] = "ok"
            break
        else:
            err = 1
    if err == 1:
        errlist.append(sender)

## 2. Escritura del csv y mostrar correos sospechosos

Ya tenemos la lista con los datos que buscábamos. Ahora lo que hemos de hacer es convertirla en un archivo .csv, en formato columna.

In [20]:
with open(f'/home/goznalo/CCC/yum-thunderbird/returns{sysargv1 + sysargv2}.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for val in okxlist:
        writer.writerow([val])

Voilà

In [22]:
####Sacar por pantalla los correos a revisar
errlist = list(sorted(dict.fromkeys(errlist)))
print("\033[1mCorreos a revisar:\033[0m")
if len(errlist) < 10 or sysargv3 == "--show":
    for err in errlist:
        print("\t" + str(err))
else:
    print('Demasiados servidores por mostrar (' + str(len(errlist)) + '). Para verlos, repetir el comando con "--show" como tercer argumento')

Correos a revisar:
	algodor.ccc.uam.es
	backupx.ccc.uam.es
	becerril.ccc.uam.es
	belvis25.ccc.uam.es
	belvis28.ccc.uam.es
	bioinfo001.ccc.uam.es
	bioinfo003.ccc.uam.es
	cibeles01.ccc.uam.es
	cibeles12.ccc.uam.es
	cibeles15.ccc.uam.es
	cibeles16.ccc.uam.es
	cibeles2-008.ccc.uam.es
	cibeles2-015.ccc.uam.es
	cibeles2-030.ccc.uam.es
	cibeles2-034.ccc.uam.es
	cibeles2-036.ccc.uam.es
	cibeles2-061.ccc.uam.es
	cibeles2-079.ccc.uam.es
	cibeles2-093.ccc.uam.es
	cibeles2-107.ccc.uam.es
	cibeles2-124.ccc.uam.es
	cibeles2-146.ccc.uam.es
	cibeles2-159.ccc.uam.es
	cibeles2-161.ccc.uam.es
	cibeles2-164.ccc.uam.es
	cibeles2-168.ccc.uam.es
	cibeles2-172.ccc.uam.es
	cibeles2-178.ccc.uam.es
	cibeles2-185.ccc.uam.es
	cibeles2-186.ccc.uam.es
	cibeles2-188.ccc.uam.es
	cibeles2-189.ccc.uam.es
	cibeles2-194.ccc.uam.es
	cibeles2-202.ccc.uam.es
	cibeles2-204.ccc.uam.es
	cibeles2-205.ccc.uam.es
	cibeles2-208.ccc.uam.es
	cibeles2-233.ccc.uam.es
	cibeles2-236.ccc.uam.es
	cibeles2-242.ccc.uam.es
	cibeles2-300.ccc.u